In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import math

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
res = requests.get("http://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=12193;type=tournament")
soup = BeautifulSoup(res.content,'lxml')

In [5]:
soup.find_all('table')[0].find_all('tbody')[0].find_all('tr')[1].find_all('td')[6].find_all('a')[0]["href"]

'/ci/engine/match/1130662.html'

###  Syed Mushtaq Ali Trophy 

In [ ]:
res = requests.get("http://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=12193;type=tournament")
soup1 = BeautifulSoup(res.content,'lxml')
numMatches = len(soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr'))

In [191]:
dfbat = pd.DataFrame() #columns={"runs_scored","balls_faced"}, index=batsman)
dfbowl = pd.DataFrame()

batsman_id = 23 #68
bowler_id = 23

for m in range(numMatches):
    if m%10==0:
        print(m)
    tmp = soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr')[m].text.split("\n")

    mlink = soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr')[m].find_all('td')[6].find_all('a')[0]["href"] 
    
    res = requests.get("http://stats.espncricinfo.com" + mlink)
    soup = BeautifulSoup(res.content,'lxml')
    
    numBatsmen = len(soup.find_all('div')[4].find_all('div')[batsman_id].find_all('div',{"class":"cell batsmen"}))

    batsman = []
    for i in range(numBatsmen):
        batsman.append(soup.find_all('div')[4].find_all('div')[batsman_id].find_all('div',{"class":"cell batsmen"})[i].text)
    df = pd.DataFrame(columns={"runs_scored","balls_faced"}, index=batsman)
    for i in range(numBatsmen):
        numRunsScored = soup.find_all('div')[4].find_all('div')[batsman_id].find_all('div',{"class":"cell runs"})[(i)*6+0].text
        numBallsFaced = soup.find_all('div')[4].find_all('div')[batsman_id].find_all('div',{"class":"cell runs"})[(i)*6+1].text 
        df.loc[batsman[i]] = pd.Series({"runs_scored":numRunsScored, "balls_faced":numBallsFaced})
    df = df.reset_index()
    df.rename(columns={"index":"player"}, inplace=True)
    df["season"] = 2018
    df["venue"] = tmp[5]
    df["date"] = tmp[6]
    df["league"] = "sma"
    dfbat = dfbat.append(df)
    
    num_bowlers = 0
    td_list = soup.find_all('div')[4].find_all('div')[23].find_all('td',{'class': None})
    for i in range(len(td_list)):
        a = td_list[i]
        b = a.find_all('span') 
        if a.text!="EXTRAS" and a.text!="TOTAL" and len(b)==0 and len(a.find_all('a'))!=0:
            num_bowlers += 1
            
    td_list = td_list[:num_bowlers*8]
    
    bowlers = []
    for i in range(num_bowlers):
        bowlers.append(td_list[i*8].text)
        
    df = pd.DataFrame(columns={"balls_bowled","runs_conceded","wickets"}, index=bowlers)
    for i in range(num_bowlers):
        numBallsBowledStr = td_list[i*8+1].text
        if len(numBallsBowledStr.split(".")) > 1:
            numBallsBowled = int(numBallsBowledStr.split(".")[0])*6 + int(numBallsBowledStr.split(".")[1])
        else:
            numBallsBowled = int(numBallsBowledStr.split(".")[0])*6
        numRunsConceded = td_list[i*8+3].text
        numWickets = td_list[i*8+4].text
        df.loc[bowlers[i]] = pd.Series({"balls_bowled":numBallsBowled, "runs_conceded":numRunsConceded, "wickets":numWickets})
    df = df.reset_index()
    df.rename(columns={"index":"player"}, inplace=True)
    df["season"] = 2018
    df["venue"] = tmp[5]
    df["date"] = tmp[6]
    df["league"] = "sma"
    dfbowl = dfbowl.append(df)
    
dfbat = dfbat[dfbat["player"]!="BATSMEN"]

0
10
20
30
40
50
60
70
80


In [195]:
dfbowl["wickets"] = dfbowl["wickets"].astype(int)
dfbowl["runs_conceded"] = dfbowl["runs_conceded"].astype(int)
dfbowl["balls_bowled"] = dfbowl["balls_bowled"].astype(int)

#dfbat = dfbat[dfbat["balls_faced"]!="-"].copy()
dfbat["balls_faced"] = dfbat["balls_faced"].astype(int)
dfbat["runs_scored"] = dfbat["runs_scored"].astype(int)

dfbat.to_csv("data/df_syed2018_bat_stats.csv")
dfbowl.to_csv("data/df_syed2018_bowl_stats.csv")

In [194]:
dfbowl

,player,wickets,runs_conceded,balls_bowled,season,venue,date,league
0,DS Kulkarni,1,29,24,2018,Rajkot,"Jan 7, 2018",sma
1,S Dube,0,39,18,2018,Rajkot,"Jan 7, 2018",sma
2,TU Deshpande,2,38,24,2018,Rajkot,"Jan 7, 2018",sma
3,SN Thakur,0,47,24,2018,Rajkot,"Jan 7, 2018",sma
4,PS Valsangkar,0,43,24,2018,Rajkot,"Jan 7, 2018",sma
5,SD Lad,0,12,6,2018,Rajkot,"Jan 7, 2018",sma
6,AA Sheth,3,31,24,2018,Rajkot,"Jan 7, 2018",sma
7,RT Arothe,0,15,12,2018,Rajkot,"Jan 7, 2018",sma
8,Swapnil Singh,0,38,18,2018,Rajkot,"Jan 7, 2018",sma
9,KH Pandya,1,48,24,2018,Rajkot,"Jan 7, 2018",sma


In [192]:
dfbat

,player,balls_faced,runs_scored,season,venue,date,league
1,KH Devdhar,24,38,2018,Rajkot,"Jan 7, 2018",sma
2,UM Patel †,28,50,2018,Rajkot,"Jan 7, 2018",sma
3,DJ Hooda (c),39,66,2018,Rajkot,"Jan 7, 2018",sma
4,KH Pandya,26,44,2018,Rajkot,"Jan 7, 2018",sma
5,Swapnil Singh,4,4,2018,Rajkot,"Jan 7, 2018",sma
7,JG Bista,25,36,2018,Rajkot,"Jan 7, 2018",sma
8,AP Tare (c) †,1,0,2018,Rajkot,"Jan 7, 2018",sma
9,SD Lad,51,82,2018,Rajkot,"Jan 7, 2018",sma
10,BR Shetty,1,0,2018,Rajkot,"Jan 7, 2018",sma
11,SA Yadav,10,22,2018,Rajkot,"Jan 7, 2018",sma


### Top batting performances

In [196]:
dft = dfbat[["player","balls_faced", "runs_scored"]].copy()
dft = dft.groupby("player").count()["balls_faced"].reset_index()
dft.rename(columns={"balls_faced":"num_innings"}, inplace=True)

df = dfbat[["player","balls_faced", "runs_scored"]].copy()
df = df.groupby(["player"]).sum().reset_index()
df = df.set_index("player").join(dft.set_index("player")).reset_index()
df["bat_sr"] = np.round(df["runs_scored"] / df["balls_faced"] * 100,1)
df["bat_avg"] = np.round(df["runs_scored"] / df["num_innings"],1)
df.sort_values("runs_scored", ascending=False)


,player,balls_faced,runs_scored,num_innings,bat_sr,bat_avg
285,RR Pant †,210,411,10,195.7,41.1
161,KH Devdhar,300,411,8,137.0,51.4
337,SP Goswami †,246,373,8,151.6,46.6
53,AS Lamba,306,335,10,109.5,33.5
329,SK Raina (c),215,314,9,146.0,34.9
202,Mandeep Singh,209,313,9,149.8,34.8
110,DR Shorey,214,303,7,141.6,43.3
190,M Vohra,214,291,9,136.0,32.3
77,B Vivek Singh,201,286,8,142.3,35.8
322,SD Lad,196,271,7,138.3,38.7


### Top bowling performances

In [198]:
dft = dfbowl[["player","runs_conceded", "balls_bowled", "wickets"]].copy()
dft = dft.groupby("player").count()["balls_bowled"].reset_index()
dft.rename(columns={"balls_bowled":"num_innings"}, inplace=True)

df = dfbowl[["player","balls_bowled", "runs_conceded", "wickets"]].copy()
df = df.groupby(["player"]).sum().reset_index()
df = df.set_index("player").join(dft.set_index("player")).reset_index()
df["bowl_sr"] = np.round(df["balls_bowled"] / df["wickets"],1)
df["bowl_econ"] = np.round(df["runs_conceded"] / df["balls_bowled"] * 6,1)
df.sort_values("wickets", ascending=False)


,player,balls_bowled,runs_conceded,wickets,num_innings,bowl_sr,bowl_econ
77,DL Chahar,203,189,19,9,10.700000,5.6
110,KK Ahmed,234,264,17,10,13.800000,6.8
191,S Aravind,164,179,15,8,10.900000,6.5
104,K Khejroliya,222,243,14,10,15.900000,6.6
12,AA Sheth,179,210,14,8,12.800000,7.0
119,LI Meriwala,148,195,13,8,11.400000,7.9
143,Mohsin Khan,162,223,13,8,12.500000,8.3
134,MK Tiwary,150,205,13,8,11.500000,8.2
160,PJ Sangwan,240,292,13,10,18.500000,7.3
247,V Athisayaraj Davidson,180,232,12,9,15.000000,7.7


### KARNATAKA PREMIER LEAGUE, 2017

In [398]:
dfp = pd.DataFrame() #columns={"runs_scored","balls_faced"}, index=batsman)

res = requests.get("http://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=11997;type=tournament")
soup1 = BeautifulSoup(res.content,'lxml')
numMatches = len(soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr'))

for m in range(numMatches):
    
    if m%10==0:
        print(m)
    tmp = soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr')[m].text.split("\n")

    mlink = soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr')[m].find_all('td')[6].find_all('a')[0]["href"] 

    res = requests.get("http://stats.espncricinfo.com" + mlink)
    soup = BeautifulSoup(res.content,'lxml')

    if len(soup.find_all('div')[4].find_all('div'))>=78:
        numBatsmen = len(soup.find_all('div')[4].find_all('div')[78].find_all('div',{"class":"cell batsmen"}))

        batsman = []
        for i in range(numBatsmen):
            batsman.append(soup.find_all('div')[4].find_all('div')[78].find_all('div',{"class":"cell batsmen"})[i].text)

        df = pd.DataFrame(columns={"runs_scored","balls_faced"}, index=batsman)
        for i in range(numBatsmen):
            numRunsScored = soup.find_all('div')[4].find_all('div')[78].find_all('div',{"class":"cell runs"})[(i)*5+0].text
            numBallsFaced = soup.find_all('div')[4].find_all('div')[78].find_all('div',{"class":"cell runs"})[(i)*5+1].text 
            df.loc[batsman[i]] = pd.Series({"runs_scored":numRunsScored, "balls_faced":numBallsFaced})

        df = df.reset_index()
        df.rename(columns={"index":"player"}, inplace=True)

        df["season"] = 2018
        df["venue"] = tmp[5]
        df["date"] = tmp[6]
        df["league"] = "kpl"
        dfp = dfp.append(df)

dfp = dfp[dfp["player"]!="BATSMEN"]
dfp2 = dfp.copy()

0
10
20


In [401]:
dfp2.head()

,player,balls_faced,runs_scored,season,venue,date,league
1,BR Sharath †,3,1,2018,Bengaluru,"Sep 1, 2017",kpl
2,M Kaunain Abbas,13,15,2018,Bengaluru,"Sep 1, 2017",kpl
3,K Gowtham,11,13,2018,Bengaluru,"Sep 1, 2017",kpl
4,KN Bharath,20,20,2018,Bengaluru,"Sep 1, 2017",kpl
5,STR Binny,25,17,2018,Bengaluru,"Sep 1, 2017",kpl


### TAMIL NADU PREMIER LEAGUE, 2017

In [404]:
dfp = pd.DataFrame() #columns={"runs_scored","balls_faced"}, index=batsman)

res = requests.get("http://stats.espncricinfo.com/tamil-nadu-premier-league-2017/engine/records/team/match_results.html?id=11901;type=tournament")
soup1 = BeautifulSoup(res.content,'lxml')
numMatches = len(soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr'))

for m in range(numMatches):
    
    if m%10==0:
        print(m)
    tmp = soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr')[m].text.split("\n")

    mlink = soup1.find_all('table')[0].find_all('tbody')[0].find_all('tr')[m].find_all('td')[6].find_all('a')[0]["href"] 

    res = requests.get("http://stats.espncricinfo.com" + mlink)
    soup = BeautifulSoup(res.content,'lxml')

    if len(soup.find_all('div')[4].find_all('div'))>=78:
        numBatsmen = len(soup.find_all('div')[4].find_all('div')[78].find_all('div',{"class":"cell batsmen"}))

        batsman = []
        for i in range(numBatsmen):
            batsman.append(soup.find_all('div')[4].find_all('div')[78].find_all('div',{"class":"cell batsmen"})[i].text)

        df = pd.DataFrame(columns={"runs_scored","balls_faced"}, index=batsman)
        for i in range(numBatsmen):
            numRunsScored = soup.find_all('div')[4].find_all('div')[78].find_all('div',{"class":"cell runs"})[(i)*5+0].text
            numBallsFaced = soup.find_all('div')[4].find_all('div')[78].find_all('div',{"class":"cell runs"})[(i)*5+1].text 
            df.loc[batsman[i]] = pd.Series({"runs_scored":numRunsScored, "balls_faced":numBallsFaced})

        df = df.reset_index()
        df.rename(columns={"index":"player"}, inplace=True)

        df["season"] = 2018
        df["venue"] = tmp[5]
        df["date"] = tmp[6]
        df["league"] = "tnpl"
        dfp = dfp.append(df)

dfp = dfp[dfp["player"]!="BATSMEN"]
dfp3 = dfp.copy()

0
10
20
30


In [405]:
dfp3.head()

,player,balls_faced,runs_scored,season,venue,date,league
1,Washington Sundar,48,69,2018,Chennai,"Jul 22, 2017",tnpl
2,KM Gandhi,33,46,2018,Chennai,"Jul 22, 2017",tnpl
3,SP Nathan,19,20,2018,Chennai,"Jul 22, 2017",tnpl
4,S Anand (c),17,26,2018,Chennai,"Jul 22, 2017",tnpl
5,Akash Sumra,3,1,2018,Chennai,"Jul 22, 2017",tnpl
